In [ ]:
import pandas as pd                                                                                                                                                                                                   
                                                                                                                                                                                                                    
df = pd.read_csv('/Users/thinhtruong/workspace/lexical-negation/results.csv')                                                                                                                                         
                                                                                                                                                                                                                    


In [ ]:
df

In [ ]:
model_dict = {'gpt2': 'GPT-2', 'cl100k_base': 'GPT-4', 'roberta-base': 'RoBERTa-base', 'google/flan-t5-xxl': 'Flan-T5-xxl',
                'google/electra-base-discriminator': 'ELECTRA-base', 'albert-base-v2': 'ALBERT', 'bert-base-uncased': 'BERT-base',
              'meta-llama/Llama-2-13b-chat-hf':'LLama-2-13B','xlnet-base-cased': 'XLNet-base'
             }

In [ ]:
df['tokenizer'] = df['tokenizer'].map(lambda x: model_dict[x])

In [ ]:
df

In [ ]:
df.to_csv('df_negmorph.csv', index = False)

In [ ]:
df_flan_over = df.loc[ (df['tokenizer'] == 'Flan-T5-xxl') & (df['NegMorph'] == 'Over-segmented') ]

In [ ]:
df_flan_over

# Error analysis
plot percentage correct/incorrect prediction for each affix for each token 


In [ ]:
df_model = df.groupby(['tokenizer', 'negative_affix'])['Predict'].mean().reset_index()

In [ ]:
df_model

In [ ]:
file_path ='/Users/thinhtruong/workspace/lexical-negation/affix_incorrect_tokenized_into.csv'                                                                                                                         
                                                                                                                                                                                                                    
# Load the dataset                                                                                                                                                                                                    
data = pd.read_csv(file_path)                                                                                                                                                                                         
                                                                                                                                                                                                                   

In [ ]:
data[data['tokenizer'] == 'LLama-2-13B']

In [ ]:
rows = []
for i, row in df.iterrows():
    tokenizer = row['tokenizer']
    word = row['neg_word']
    negative_affix = row['negative_affix']
    # print(word, tokenizer)
    row_tmp = data.loc[ (data['tokenizer'] == tokenizer) & (data['neg_word'] == word)]
    tokenized_into = row_tmp['tokenized_into'].array
    if len(tokenized_into):
        tokenized_into = tokenized_into[0]
    else:
        tokenized_into = negative_affix
    # print(tokenized_into)
    row['tokenized_into'] = tokenized_into
    
    rows.append(row)
df_merged = pd.DataFrame(data = rows)

In [ ]:
df_merged

In [ ]:
import numpy as np

In [ ]:
df_merged['Counts'] = np.zeros(len(df_merged))


In [ ]:
df_merged_by_model = df_merged.groupby(['tokenizer', 'negative_affix', 'tokenized_into', 'Predict'])["Predict"].count().reset_index(name = "count")                                                                       

In [ ]:
df_merged_by_model

In [ ]:
df_merged_by_model_total = df_merged_by_model.groupby(['tokenizer', 'negative_affix'])['count'].sum().reset_index(name = "total")

In [ ]:
rows = []
for i, row in df_merged_by_model.iterrows():
    row['total'] = df_merged_by_model_total[df_merged_by_model_total['negative_affix'] == row['negative_affix']]['total'].iloc[0]
    row['NegMorph'] = df_merged.loc[(df_merged['negative_affix'] == row['negative_affix']) & (df_merged['tokenized_into'] == row['tokenized_into'])]['NegMorph'].iloc[0] 
    rows.append(row)
df_merged_by_model_total = pd.DataFrame(data = rows)

In [ ]:
df_merged_by_model_total

In [ ]:
df_merged_by_model_total_negmorph = df_merged_by_model_total.groupby(['tokenizer', 'negative_affix','NegMorph','Predict'])['count'].sum().reset_index(name = "NegMorph_count")

In [ ]:
df_merged_by_model_total_negmorph_percentage = df_merged_by_model_total_negmorph.groupby(['tokenizer','negative_affix','NegMorph'])['NegMorph_count'].sum().reset_index(name = 'total')

In [ ]:
df_merged_by_model_total_negmorph_percentage

#draw here
percentage instead of count
1 | 0
for each 1: negmorph (percentage) negmorph_count/ total for that negative affix
max 6 values (3 negmorph x 2 predict) for each


In [ ]:
df_merged_by_model_total_negmorph.to_csv('affix_performance_by_negmorph.csv', index = False)

In [ ]:
df_merged_by_model_total_negmorph = pd.read_csv('affix_performance_by_negmorph.csv')

In [ ]:
df_merged_by_model_total_negmorph

In [ ]:
df_merged_by_model_total_affix = df_merged_by_model_total_negmorph.groupby(['tokenizer', 'negative_affix'])['NegMorph_count'].sum().reset_index(name = 'affix_count')

In [ ]:
df_merged_by_model_total_affix

In [ ]:
rows = []
for i, row in df_merged_by_model_total_negmorph.iterrows():
    affix = row['negative_affix']
    tokenizer = row['tokenizer']
    total = df_merged_by_model_total_affix.loc[(df_merged_by_model_total_affix['tokenizer'] == tokenizer) & (df_merged_by_model_total_affix['negative_affix'] == affix)]['affix_count'].iloc[0]
    # print(total)
    row['percentage'] = int(row['NegMorph_count']) / int(total)
    # print(row['percentage'])
    row['total'] = total

    rows.append(row)
    # print(row)
    # print('================')
# print(rows)
df_merged_percentage = pd.DataFrame(data = rows)

In [ ]:
df_merged_percentage

In [ ]:
df_merged_percentage.to_csv('affix_negmorph_percentage.csv', index = False)

In [ ]:
df_merged_percentage = pd.read_csv('affix_negmorph_percentage.csv')

In [ ]:
df_merged_percentage

In [ ]:
df_merged_percentage.groupby(['tokenizer','negative_affix'])['percentage'].sum()

In [ ]:
top_affixes_df = df_merged_percentage.sort_values('total', ascending=False).drop_duplicates('negative_affix').head(40)                                                                                                                  
                                                                                                                                                                                                                    
# Now we need to filter the original DataFrame to only include rows that correspond to the top 10 affixes                                                                                                             
top_affixes_list = top_affixes_df['negative_affix'].tolist()                                                                                                                                                          
filtered_df = df_merged_percentage[df_merged_percentage['negative_affix'].isin(top_affixes_list)]                                                                                                                                                         
                                                                                                                                                                                                                    
# Display the filtered DataFrame to verify the selection                                                                                                                                                              
filtered_df

In [ ]:
top_affixes_pivot = filtered_df.pivot_table(index='negative_affix', columns=['tokenizer','Predict','NegMorph'], values='percentage', fill_value=0)

In [ ]:
top_affixes_pivot

In [ ]:
top_affixes_pivot.loc['mean'] = top_affixes_pivot.mean()

In [ ]:
top_affixes_pivot

In [ ]:
gpt4_dict = {'model': 'GPT-4'}
flan_dict = {'model': 'Flan-T5-xxl'}
llama_dict = {'model': 'Llama-2-13B'}


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches


In [ ]:
tokenizers = filtered_df['tokenizer'].unique()  
# print(tokenizers)
tokenizers = ['GPT-4','Flan-T5-xxl','LLama-2-13B']                                                                                                                                                                                                                        
# Create a figure with subplots for each tokenizer                                                                                                                                                                    
fig, axes = plt.subplots(nrows=1, ncols=len(tokenizers), figsize=(10 * len(tokenizers), 15), sharex= True)                                                                                                              
                                                                                                                                                                                                                    
# Plot a stacked bar plot for each tokenizer                                                                                                                                                                          
for ax, tokenizer in zip(axes, tokenizers):                                                                                                                                                                           
  # Filter the DataFrame for the current tokenizer                                                                                                                                                                  
    tokenizer_df = filtered_df[filtered_df['tokenizer'] == tokenizer]                                                                                                                         
    # Pivot the data for the current tokenizer                                                                                                                                                                        
    pivot_df = tokenizer_df.pivot_table(index='negative_affix',                                                                                                                                                       
                                      columns=['Predict', 'NegMorph'],                                                                                                                                              
                                      values='percentage',                                                                                                                                                          
                                      fill_value=0)
    pivot_df.loc['Overall'] = pivot_df.mean()
    pivot_df = pivot_df[::-1]
    if tokenizer == 'GPT-4':
        gpt4_dict['NegMorph_Correct'] = pivot_df.loc['Overall'][0][0]
        gpt4_dict['NegMorph_Over-segmented'] = pivot_df.loc['Overall'][0][1]
        gpt4_dict['NegMorph_Under-segmented'] = pivot_df.loc['Overall'][0][2]
    elif tokenizer == 'Flan-T5-xxl':
        flan_dict['NegMorph_Correct'] = pivot_df.loc['Overall'][0][0]
        flan_dict['NegMorph_Over-segmented'] = pivot_df.loc['Overall'][0][1]
        flan_dict['NegMorph_Under-segmented'] = pivot_df.loc['Overall'][0][2]
    elif tokenizer == 'LLama-2-13B':
        llama_dict['NegMorph_Correct'] = pivot_df.loc['Overall'][0][0]
        llama_dict['NegMorph_Over-segmented'] = pivot_df.loc['Overall'][0][1]
        llama_dict['NegMorph_Under-segmented'] = pivot_df.loc['Overall'][0][2]

        
    print(pivot_df.loc['Overall'][0][0])
    # print(pivot_df)
  # Plot each part of the bar for Predict 0 and 1                                                                                                                                                                   
    colors = {'Correct':'g', 'Under-segmented': 'y', 'Over-segmented': 'r'}
    patterns = {0: '/', 1:'x'}
    edges = {0: 'w' , 1: 'black'}
    for predict in [0, 1]:
        bottom = [0] * len(pivot_df)  # Initialize the bottom at 0 for each affix
        for i,negmorph in enumerate(pivot_df.columns.levels[1]):
            if predict == 1:
                width = -pivot_df[(predict, negmorph)]
                ax.barh(pivot_df.index, width = width, left=bottom,                                                                                                                                      
                     label=negmorph, color = colors[negmorph], alpha = 0.6, edgecolor = 'white')                                                                                                                                                           
                
            else:
                width = pivot_df[(predict, negmorph)]
                ax.barh(pivot_df.index, width = width, left=bottom,                                                                                                                                      
                     label=negmorph, color = colors[negmorph], alpha = 1, edgecolor = 'white')                                                                                                                                                           

            # print(pivot_df.index)
            # Update the bottom for the next NegMorph                                                                                                                                                                 
            bottom += width.fillna(0)                                                                                                                                                         
            # print(bottom)
    bars = ax.patches
    # print(len(bars))
    for i in range(0, 246, 41):
        bars[i].set_hatch('/')
        bars[i].set_edgecolor('black')
    # for i in range(42, 246, 41):
    #     bars[i].set_color('x')
    # Set the title for the current subplot                                                                                                                                                                           
    ax.set_title(tokenizer)
    ax.set_title(f'{tokenizer}', fontsize=30)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.tick_params(axis='y', which='major', labelsize=25)

    # break
                                                                                                                                                                                                                    
# Set common labels                                                                                                                                                                                                   
axes[1].set_xlabel('Percentage', fontsize=25) 
# ax.set_ylabel('Negative Affix')                                                                                                                                                                                       

                                                                                                                                                                                                                    
# Rotate x-axis labels for better readability                                                                                                                                                                         
for ax in axes:                                                                                                                                                                                                       
    ax.set_xticklabels(['1','','0.5','','0','','0.5','','1'], fontsize = 20)

                                                                                                                                                                                                                    
# Add legend                                                                                                                                                                                                          
red_patch = mpatches.Patch(color='r', label='Over-segmented')
yellow_patch = mpatches.Patch(color='y', label='Under-segmented')
green_patch = mpatches.Patch(color='g', label='Correct')

axes[1].legend(handles=[green_patch, yellow_patch,red_patch],fontsize=25, loc = 'upper center', bbox_to_anchor=(0.5, -0.05), ncol=3)
# plt.axis('off')
# plt.xticks([])
                                                                                                                                                                                                        
# Save the plot as an image                                                                                                                                                                                           
plt.tight_layout()
plt.show()
fig.savefig('affix_negmorph_distribution_by_tokenizer.pdf', bbox_inches='tight')

In [ ]:
gpt4_dict

Correlation

In [ ]:
import seaborn as sns

In [ ]:
# df_all: combine all metric to compare: neg non-neg all, 3 negmorph, 3 models(?) or overall

In [ ]:
# load accuracy into df_accuracy
gpt4_pred = pd.read_csv('gpt4_pred.csv')
flan_pred = pd.read_csv('flant5_pred.csv')
llama2_pred = pd.read_csv('llama2_pred.csv')

In [ ]:
gpt4_pred.head()

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
gpt4_accuracy = accuracy_score(gpt4_pred['gold'], gpt4_pred['pred'])

In [ ]:
def calculate_acc(df):
    df_neg = df[df['gold'] == 0]
    acc_neg = accuracy_score(df_neg['gold'], df_neg['pred'])
    df_pos = df[df['gold'] == 1]
    acc_pos = accuracy_score(df_pos['gold'], df_pos['pred'])
    acc = accuracy_score(df['gold'], df['pred'])
    return {'acc_neg':acc_neg, 'acc_pos':acc_pos, 'acc_all':acc}

In [ ]:
results_dict = []

In [ ]:
gpt4_dict.update(calculate_acc(gpt4_pred))

flan_dict.update(calculate_acc(flan_pred))
# flan_dict['model'] = 'FLan-T5-xxl'
llama_dict.update(calculate_acc(llama2_pred))
# llama_dict['model'] = 'Llama-2-13B'

In [ ]:
gpt4_dict

In [ ]:
results = [gpt4_dict, flan_dict, llama_dict]

In [ ]:
df_all = pd.DataFrame(data = results)

In [ ]:
df_all

In [ ]:
df_all

In [ ]:
acc_metrics = ['acc_neg']
negmorph_metrics = ['NegMorph_Correct', 'NegMorph_Over-segmented', 'NegMorph_Under-segmented']

In [ ]:
from scipy.stats import pearsonr, spearmanr, kendalltau
import numpy as np
from scipy import stats

In [ ]:
from scipy.stats import bootstrap

In [ ]:
def pearson_func(x,y):
    return pearsonr(x,y)[0]

In [ ]:
rng = np.random.default_rng()

In [ ]:
def corrfunc(x, y, hue=None, ax=None, **kws):
    """Plot the correlation coefficient in the top left hand corner of a plot."""
    try:
        # print(method)
        # method = stats.PermutationMethod(n_resamples=np.inf, random_state=rng)
        # method = stats.MonteCarloMethod(rvs=(rng.uniform, rng.uniform))

        r, p = pearsonr(x, y)
        print(r,p)
    except:
        r = -1
        p = 0.0
    # print(r,p)
    # res = bootstrap((x,y), pearson_func , vectorized = False, paired = True)
    # print(res.confidence_interval)
    # print(res)
    # method = stats.BootstrapMethod(method='BCa', random_state=rng)
    # print(pearsonr(x, y).confidence_interval(confidence_level=0.9, method=method))
    if p <= 0.05:
        add_asterisk = '*'
    else:
        add_asterisk = ''
    ax = ax or plt.gca()
    ax.annotate(f'r = {r:.3f}{add_asterisk}', xy=(0.1, 0.97), xycoords=ax.transAxes)

sns.set(font_scale=1)
sns.set_style("whitegrid", {'axes.grid': False})

g = sns.pairplot(
    df_all, 
    # diag_kind="kde",
    y_vars = acc_metrics,
    x_vars = negmorph_metrics,
    kind="reg", 
    plot_kws={
        'line_kws':{'color':'red'},
        'scatter_kws': {'s': 2}
    },
    # corner = True,
    height = 2
    # hue = 'category'
    
)
g.map(corrfunc)

acc_metrics_label = ['Neg']
negmorph_metrics_label = ['Correct', 'Over-segmented', 'Under-segmented']

for i in range(len(negmorph_metrics)):
    for j in range(len(acc_metrics)):
        g.axes[j,i].xaxis.set_label_text(negmorph_metrics_label[i])
        g.axes[j,i].yaxis.set_label_text(acc_metrics_label[j])

# g.set( ylabel = 'Accuracy')

g.tight_layout()
# g.axes[0,0].set_xlim((0.0,0.8))
# g.axes[0,1].set_xlim((-0.1,1.0))
# g.axes[0,2].set_xlim((-0.1,1.0))
# g.axes[0,3].set_xlim((-0.1,1.0))
# g.axes[0,4].set_xlim((-0.1,1.0))

plt.show()